## 

In [66]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")

Imported!


In [67]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
data = BeautifulSoup(extracting_data, 'lxml')

In [68]:
Columns = ['Postalcode', 'Borough', 'Neighborhood']
toronto_df = pd.DataFrame(columns = Columns)
content = data.find('table', class_= 'wikitable sortable')
tbody = content.tbody  

In [80]:
toronto_df = pd.DataFrame(columns = toronto_df.columns)

In [81]:
dict = {}
for tr in tbody.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        if i == 0:
            dict['Postalcode'] = td.text.strip('\n')
            i += 1
        elif i==1 :
            dict['Borough'] = td.text.strip('\n')
            i += 1
        elif i == 2:
            dict['Neighborhood'] = td.text.strip('\n')
    if len(dict) != 0:    
        toronto_df = toronto_df.append(dict, ignore_index=True)
toronto_df.head()  

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop cells with a borough that is "Not assigned"

In [82]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned'].reset_index(drop=True)
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Group neighborhoods in the same Borough

In [83]:
toronto_df = toronto_df.replace(r'/',', ', regex=True)
toronto_df

# toronto_df= toronto_df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
# toronto_df

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### For Neighborhood="Not assigned", make the value the same as Borough

In [84]:
for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [85]:
toronto_df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6B,North York,Downsview
freq,1,24,4


In [86]:
toronto_df.shape

(103, 3)

In [87]:
toronto_df.to_csv("CapstoneCanadaPre.csv")